In [20]:
# Importamos librerías necesarias
import telebot
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

In [ ]:
# Variables de configuración
TOKEN = ''
CSV_FILE = '..\data\processed\Articulos_LLM2.csv'  # ← tu archivo CSV

# Cargamos el CSV de artículos
articulos_df = pd.read_csv(CSV_FILE)


# Preparamos los textos para analizar
# Vamos a unir titulo + contenido + fechas + precios + edad
articulos_df['texto_completo'] = (
    articulos_df['titulo'].fillna('') + '. ' +
    articulos_df['contenido'].fillna('') + ' ' +
    articulos_df['fechas_contexto'].fillna('') + ' ' +
    articulos_df['Precios'].fillna('') + ' '
)

In [23]:
import nltk
from nltk.corpus import stopwords

# Asegúrate de tener descargado el paquete de stopwords de NLTK
nltk.download('stopwords')

# Cargamos las stopwords en español
stopwords_es = stopwords.words('spanish')

# Creamos el TF-IDF usando las stopwords en español
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords_es)

# Ahora sí, transformamos
tfidf_matrix = tfidf_vectorizer.fit_transform(articulos_df['texto_completo'])


[nltk_data] Downloading package stopwords to C:\Users\Abdon.RAMIREZ-
[nltk_data]     BRICEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from datetime import datetime, timedelta
import dateparser
import re

# Inicializamos el bot
bot = telebot.TeleBot(TOKEN)

# --- Nueva función: interpretar fechas en la pregunta ---

def interpretar_fecha_usuario(pregunta):
    pregunta = pregunta.lower()
    hoy = datetime.now()

    if 'hoy' in pregunta:
        return hoy.date(), hoy.date()
    
    elif 'mañana' in pregunta:
        return (hoy + timedelta(days=1)).date(), (hoy + timedelta(days=1)).date()
    
    elif 'semana que viene' in pregunta or 'semana próxima' in pregunta or 'semana siguiente' in pregunta:
        inicio = hoy + timedelta(days=(7 - hoy.weekday()))  # lunes próximo
        fin = inicio + timedelta(days=6)  # domingo de la próxima semana
        return inicio.date(), fin.date()
    
    elif 'esta semana' in pregunta:
        # Desde hoy hasta domingo
        inicio = hoy
        fin = hoy + timedelta(days=(6 - hoy.weekday()))
        return inicio.date(), fin.date()

    elif 'fin de semana' in pregunta or 'sábado' in pregunta or 'domingo' in pregunta:
        # Calcula el próximo sábado y domingo
        dias_hasta_sabado = (5 - hoy.weekday()) % 7
        sabado = hoy + timedelta(days=dias_hasta_sabado)
        domingo = sabado + timedelta(days=1)
        return sabado.date(), domingo.date()
    
    else:
        # No se detectó ninguna fecha relevante
        return None, None


# --- Nueva función: convertir fechas del dataset ---
def preparar_fechas_dataset(df):
    fecha_inicio_parseada = []
    fecha_fin_parseada = []

    for inicio, fin in zip(df['fecha_inicio'].fillna(''), df['fecha_fin'].fillna('')):
        # Parsear fecha inicio
        try:
            if inicio:
                fecha_inicio_dt = dateparser.parse(str(inicio), languages=['es'])
                fecha_inicio_parseada.append(fecha_inicio_dt.date() if fecha_inicio_dt else None)
            else:
                fecha_inicio_parseada.append(None)
        except Exception as e:
            fecha_inicio_parseada.append(None)

        # Parsear fecha fin
        try:
            if fin:
                fecha_fin_dt = dateparser.parse(str(fin), languages=['es'])
                fecha_fin_parseada.append(fecha_fin_dt.date() if fecha_fin_dt else None)
            else:
                fecha_fin_parseada.append(None)
        except Exception as e:
            fecha_fin_parseada.append(None)

    df['fecha_inicio_parseada'] = fecha_inicio_parseada
    df['fecha_fin_parseada'] = fecha_fin_parseada

    return df# Procesamos el dataframe para agregar la columna de fechas
articulos_df = preparar_fechas_dataset(articulos_df)

# --- Función para buscar la mejor respuesta ---
def buscar_respuesta(pregunta_usuario):
    fecha_inicio_consulta, fecha_fin_consulta = interpretar_fecha_usuario(pregunta_usuario)

    if fecha_inicio_consulta and fecha_fin_consulta:
        print(f"DEBUG 🛠 Fecha detectada -> Inicio: {fecha_inicio_consulta}, Fin: {fecha_fin_consulta}")
    else:
        print("DEBUG 🛠 No se detectó ninguna fecha en la consulta.")

    if fecha_inicio_consulta and fecha_fin_consulta:
        filtrados = articulos_df[
            (articulos_df['fecha_inicio_parseada'].notna()) &
            (articulos_df['fecha_fin_parseada'].notna()) &
            (articulos_df['fecha_inicio_parseada'] <= fecha_fin_consulta) &
            (articulos_df['fecha_fin_parseada'] >= fecha_inicio_consulta)
        ]

        if filtrados.empty:
            return "🤔 No encontré actividades activas para esas fechas. ¡Intenta preguntar otra cosa!"
    else:
        filtrados = articulos_df

    if filtrados.empty:
        return "🤔 No encontré actividades. ¡Intenta preguntar otra cosa!"

    # TF-IDF búsqueda de texto
    tfidf_matrix_filtrada = tfidf_vectorizer.transform(filtrados['texto_completo'])
    pregunta_vectorizada = tfidf_vectorizer.transform([pregunta_usuario])
    similitudes = cosine_similarity(pregunta_vectorizada, tfidf_matrix_filtrada)

    idx_mejor = np.argmax(similitudes)
    mejor_score = similitudes[0, idx_mejor]

    print(f"DEBUG 🛠 Mejor score de similitud: {mejor_score}")

    # Cambiamos el threshold de 0.2 a 0.05
    if mejor_score < 0.05:
        # Si no hay suficiente similitud, igual respondemos el primer evento disponible
        articulo = filtrados.iloc[0]
        respuesta = (
            f"🔎 No encontré un evento perfectamente relacionado con tu pregunta, pero te recomiendo:\n\n"
            f"✅ Actividad: *{articulo['titulo']}*\n\n"
            f"{articulo['contenido'][:400]}...\n\n"
            f"💰 Precio: {articulo['Precios']}\n"
            f"🔗 Más info: {articulo['url']}"
        )
        return respuesta
    else:
        articulo = filtrados.iloc[idx_mejor]
        respuesta = (
            f"✅ Actividad encontrada: *{articulo['titulo']}*\n\n"
            f"{articulo['contenido'][:400]}...\n\n"
            f"💰 Precio: {articulo['Precios']}\n"
            f"🔗 Más info: {articulo['url']}"
        )
        return respuesta



# --- Manejadores del bot ---
@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(
        message.chat.id,
        f"¡Hola, {message.from_user.first_name}! 👋 Soy tu asistente virtual. "
        "Pregúntame qué actividades hay disponibles para hoy, mañana o la próxima semana. 🎯"
    )

@bot.message_handler(func=lambda message: True)
def responder(message):
    consulta = message.text
    respuesta = buscar_respuesta(consulta)
    bot.send_message(message.chat.id, respuesta, parse_mode='Markdown')

# Lanzamos el bot
print("🤖 Bot activo y esperando mensajes...")
bot.polling()


🤖 Bot activo y esperando mensajes...
DEBUG 🛠 Fecha detectada -> Inicio: 2025-05-03, Fin: 2025-05-04
DEBUG 🛠 Mejor score de similitud: 0.08360930195154836
DEBUG 🛠 Fecha detectada -> Inicio: 2025-04-28, Fin: 2025-04-28
DEBUG 🛠 Mejor score de similitud: 0.1327516682593147
DEBUG 🛠 No se detectó ninguna fecha en la consulta.
DEBUG 🛠 Mejor score de similitud: 0.28457168430400187


In [26]:
articulos_df['fecha_fin_parseada'].head(25)

0     2025-05-15
1     2025-04-27
2     2025-05-11
3           None
4     2025-04-30
5     2025-06-15
6           None
7           None
8           None
9           None
10          None
11    2025-05-02
12          None
13    2025-05-03
14    2025-05-04
15    2025-05-15
16    2025-05-24
17    2025-04-26
18    2025-05-30
19          None
20          None
21          None
22          None
23          None
24          None
Name: fecha_fin_parseada, dtype: object